In [ ]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
from bbmodcache.bbSearch import SearchProblem, search

In [ ]:
class Robot:
    def __init__(self, location, carried_items, strength):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength

    def weight_carried(self):
        return sum([ITEM_WEIGHT[i] for i in self.carried_items])

    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location,
                      self.carried_items,
                      self.strength ) )

class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}

    ## Define a unique string representation for a door object
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

In [ ]:
class State:
    def __init__( self, robot, doors, room_contents ):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents

    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )


In [ ]:
ROOM_CONTENTS = {
    'cemetery of ash'     : {'coiled sword'},
    'firelink shrine'     : {'estus flask'},
    'locked tower'        : {'uchiagatana'},
    'high wall of lothric': {'lothric banner', 'cell key'},
    'undead settlement'   : {'transposing kiln'},
    'farron keep'         : {'soul of the blood of the wolf'},
}

ITEM_WEIGHT = {
                 'coiled sword' : 3,
                  'estus flask' : 3,
                  'uchiagatana' : 3,
               'lothric banner' : 4,
                     'cell key' : 1,
             'transposing kiln' : 3,
'soul of the blood of the wolf' : 5,
}

DOORS = [
    Door( 'cemetery of ash', 'firelink shrine' ),
    Door( 'firelink shrine', 'high wall of lothric' ),
    Door( 'firelink shrine', 'locked tower', doorkey='cell key', locked=False ),
    Door( 'high wall of lothric', 'undead settlement' ),
    Door( 'undead settlement', 'farron keep' ),
]


In [ ]:
from copy import deepcopy

class RobotWorker( SearchProblem ):

    def __init__( self, state, goal_item_locations ):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations

    def possible_actions( self, state ):

        robot_location = state.robot.location
        strength       = state.robot.strength
        weight_carried = state.robot.weight_carried()

        actions = []
        # Can put down any carried item
        for i in state.robot.carried_items:
            actions.append( ("put down", i) )

        # Can pick up any item in room if strong enough
        for i in state.room_contents[robot_location]:
            if strength >= weight_carried + ITEM_WEIGHT[i]:
                actions.append( ("pick up", i))

        # If there is an unlocked door between robot location and
        # another location can move to that location
        for door in state.doors:
            if  door.locked==False and robot_location in door.goes_between:
                actions.append( ("move to", door.other_loc[robot_location]) )

        # Now the actions list should contain all possible actions
        return actions

    def successor( self, state, action):
        next_state = deepcopy(state)
        act, target = action
        if act== "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)

        if act == "pick up":
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)

        if act == "move to":
            next_state.robot.location = target

        return next_state

    def goal_test(self, state):
        #print(state.room_contents)
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    return False
        return True

    def display_state(self,state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.carried_items)
        print("Room contents:", state.room_contents)

In [ ]:
rob = Robot('cemetery of ash', [], 12 )

state = State(rob, DOORS, ROOM_CONTENTS)

goal_item_locations =  {"firelink shrine":{"soul of the blood of the wolf", "transposing kiln", "coiled sword"},
                        "undead settlement":{"lothric banner"},
                        "farron keep":{"uchiagatana"}}

RW_PROBLEM_1 = RobotWorker( state, goal_item_locations )

In [ ]:
poss_acts = RW_PROBLEM_1.possible_actions( RW_PROBLEM_1.initial_state )
poss_acts

In [ ]:
for act in poss_acts:
    print("Action", act, "leads to the following state:")
    next_state = RW_PROBLEM_1.successor( RW_PROBLEM_1.initial_state, act )
    RW_PROBLEM_1.display_state(next_state)
    print()

In [ ]:
search( RW_PROBLEM_1, 'DF/LIFO', 100000, loop_check=True)